In [563]:

import plotly.graph_objects
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
# from pandas.tseries.offsets import *


from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) 
import plotly.express as px
import plotly.graph_objects as go

import loguru
import requests

pandas.set_option('display.max_rows', 500)
# import sys
# sys.path.insert(1, '/Users/singularity/Aaron/coding/fintech/OptionsDailyInfoData')

In [564]:
TAIEX_VALUE = 0

year = 2018
month = 8
day = 13

In [565]:
filenameTaiexs = f'taiexs-2019.csv'
filepathTaiexs = f'/Users/singularity/Aaron/coding/fintech/{filenameTaiexs}'

dfTaiexs = pandas.read_csv(filepathTaiexs)
dfTaiexs['Date'] = pandas.to_datetime(dfTaiexs['Date'], format='%Y%m%d')

TAIEX_VALUE = float(dfTaiexs[dfTaiexs['Date'] == f'{year}-{month:02}-{day:02}']['ClosePrice'])
TAIEX_VALUE_BOTTOM = int(TAIEX_VALUE - (TAIEX_VALUE % 50))
TAIEX_VALUE_TOP = TAIEX_VALUE_BOTTOM + 50

print(f'TAIEX_VALUE: {TAIEX_VALUE}')
print(f'TAIEX_VALUE_BOTTOM: {TAIEX_VALUE_BOTTOM}')
print(f'TAIEX_VALUE_TOP: {TAIEX_VALUE_TOP}')

TAIEX_VALUE: 10748.92
TAIEX_VALUE_BOTTOM: 10700
TAIEX_VALUE_TOP: 10750


In [566]:
print(f'{year}-{month:02}-{day:02}')
dfTaiexs[(dfTaiexs['Date'] >= (datetime.datetime(year, month, day)-datetime.timedelta(days=5)))& (dfTaiexs['Date'] <= datetime.datetime(year, month, day)+datetime.timedelta(days=5))]
# dfTaiexs[dfTaiexs['Date'] >= datetime(year, month, day-7)]['ClosePrice']

2018-08-13


,Date,OpenPrice,HighestPrice,LowestPrice,ClosePrice
47,2018-08-08,11026.07,11095.50,11026.07,11075.25
48,2018-08-09,11063.62,11063.62,11001.96,11028.07
49,2018-08-10,11009.96,11033.89,10972.25,10983.68
50,2018-08-13,10939.68,10939.68,10693.73,10748.92
51,2018-08-14,10797.95,10829.85,10755.53,10824.23
52,2018-08-15,10827.28,10827.28,10688.59,10716.75
53,2018-08-16,10663.57,10725.71,10606.26,10683.90
54,2018-08-17,10702.60,10757.01,10688.55,10690.96


In [567]:

filename = f'optionsDailyInfo-{year}{month:02}{day:02}.csv'
filepath = f'/Users/singularity/Aaron/coding/fintech/OptionsDailyInfoData/{filename}'
df = pandas.read_csv(filepath)
df

,ContractMonthWeek,StrikePrice,CallPut,Open,High,Low,Close,Volume,OpenInterest,BestBid,BestAsk,HistoricalHigh,HistoricalLow,Session
0,201808,9000.0,Call,-,-,-,-,0,5,-,-,-,-,Regular
1,201808,9000.0,Call,-,-,-,-,0,-,-,-,-,-,AfterHours
2,201808,9000.0,Put,0.2,0.2,0.2,0.2,2,4770,0.1,0.2,20,0.1,Regular
3,201808,9000.0,Put,0.2,0.2,0.2,0.2,4,-,0.1,1.1,20,0.1,AfterHours
4,201808,9100.0,Call,-,-,-,-,0,0,-,-,-,-,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,201903,13200.0,Put,-,-,-,-,0,-,-,-,-,-,AfterHours
772,201903,13400.0,Call,5.7,5.7,5.7,5.7,1,803,1.6,54,5.7,5.1,Regular
773,201903,13400.0,Call,-,-,-,-,0,-,-,-,-,-,AfterHours
774,201903,13400.0,Put,-,-,-,-,0,0,-,-,-,-,Regular


In [568]:
df.astype({
    "ContractMonthWeek": "str"}).dtypes
# df.astype({
#     "ContractMonthWeek": "string",
#     "StrikePrice": "int64",
#     "Open": "float64",
#     "High": "float64",
#     "Low": "float64",
#     "Close": "float64",
#     "Volume": "int64",
#     "OpenInterest": "int64",
#     "BestBid": "float64",
#     "BestAsk": "float64",
#     "HistoricalHigh": "float64",
#     "HistoricalLow": "float64"}, errors='ignore').dtypes

ContractMonthWeek     object
StrikePrice          float64
CallPut               object
Open                  object
High                  object
Low                   object
Close                 object
Volume                 int64
OpenInterest          object
BestBid               object
BestAsk               object
HistoricalHigh        object
HistoricalLow         object
Session               object
dtype: object

In [569]:

# df.head(200)
ContractMonthWeek = df["ContractMonthWeek"][0]
# ContractMonthWeek = "201807"

StrikePriceLimite = df["StrikePrice"].apply(lambda d: True if int(d)>=TAIEX_VALUE-150 and int(d) <=TAIEX_VALUE+150 else False)
dfCall = df[(df["CallPut"]=="Call") & (df["Session"]=="Regular") & (df["ContractMonthWeek"]==ContractMonthWeek) & StrikePriceLimite][['ContractMonthWeek', 'StrikePrice','CallPut', 'Close']]
dfPut = df[(df["CallPut"]=="Put") & (df["Session"]=="Regular") & (df["ContractMonthWeek"]==ContractMonthWeek) & StrikePriceLimite][['ContractMonthWeek', 'StrikePrice','CallPut', 'Close']]
# df.loc[StrikePriceLimite]

In [570]:
dfCall.head()

,ContractMonthWeek,StrikePrice,CallPut,Close
64,201808,10600.0,Call,171
68,201808,10700.0,Call,91
72,201808,10750.0,Call,56
76,201808,10800.0,Call,31
80,201808,10850.0,Call,14


In [571]:
print(f'TAIEX_VALUE: {TAIEX_VALUE}')
print(f'TAIEX_VALUE_BOTTOM: {TAIEX_VALUE_BOTTOM}')
print(f'TAIEX_VALUE_TOP: {TAIEX_VALUE_TOP}\n')
optionsDF = pandas.merge(dfCall, dfPut, left_on=["StrikePrice", 'ContractMonthWeek'], right_on=["StrikePrice",      'ContractMonthWeek']\
            , suffixes=('_Call','_Put'))[['Close_Call', 'CallPut_Call', 'StrikePrice', 'CallPut_Put', 'Close_Put']]
optionsDF

TAIEX_VALUE: 10748.92
TAIEX_VALUE_BOTTOM: 10700
TAIEX_VALUE_TOP: 10750



,Close_Call,CallPut_Call,StrikePrice,CallPut_Put,Close_Put
0,171,Call,10600.0,Put,17
1,91,Call,10700.0,Put,35.5
2,56,Call,10750.0,Put,51
3,31,Call,10800.0,Put,75
4,14,Call,10850.0,Put,110


In [572]:
optionsDF.dtypes
optionsDF.astype({
    "Close_Call": "float64",
    "CallPut_Call": "string",
    "StrikePrice": "int64",
    "CallPut_Put": "string",
    "Close_Put": "float64"}, errors='ignore').dtypes

Close_Call      float64
CallPut_Call     string
StrikePrice       int64
CallPut_Put      string
Close_Put       float64
dtype: object

In [573]:
# buyCallStrikePrice = 11500
# sellCallStrikePrice = 0

# buyCall = float(optionsDF[optionsDF["StrikePrice"] == buyCallStrikePrice]["Close_Call"])
# sellCall = optionsDF[optionsDF["StrikePrice"] == sellCallStrikePrice]["Close_Call"]

# buyPutStrikePrice = 11500.0
# sellPutStrikePrice = 0

# buyPut = optionsDF[optionsDF["StrikePrice"] == buyPutStrikePrice]["Close_Put"]
# sellPut = optionsDF[optionsDF["StrikePrice"] == sellPutStrikePrice]["Close_Put"]
# buyCall

In [574]:
def optionsProfit(x, strikePrice, optionsDF, PutCall, BuySell):
    # y = np.array([0 for _ in range(len(x))])
    y = [0 for _ in range(len(x))]
    
    t = -1 if BuySell == "buy" else 1
    s = -50*t
    if PutCall == "call":
        price = float(optionsDF[optionsDF["StrikePrice"] == strikePrice]["Close_Call"])*t
        for i in range(len(x)):
            if x[i] <= strikePrice:
                y[i] = price
            else:
                price += s
                y[i] = price
    else:
        price = float(optionsDF[optionsDF["StrikePrice"] == strikePrice]["Close_Put"])*t
        for i in range(len(x)-1,-1,-1):
            if x[i] >= strikePrice:
                y[i] = price
            else:
                price += s
                y[i] = price
    return y

In [583]:
class options:
    def __init__(
        self,
        strikePrice,
        optionsDF,
        PutCall,
        BuySell
        ):
        self.strikePrice = strikePrice
        self.optionsDF = optionsDF
        self.PutCall = PutCall
        self.BuySell = BuySell
    def optionsProfit(self, x):
        Exchange_Fee = -1
        y = [0 for _ in range(len(x))]
        
        t = -1 if self.BuySell == "buy" else 1
        s = -50*t
        if self.PutCall == "call":
            print(f'@@@@optionsDF\n{self.optionsDF}')
            print(f'@@@@StrikePrice\n{self.strikePrice}')
            print(f'@@@@price\n{self.self.optionsDF[self.optionsDF["StrikePrice"] == self.strikePrice]["Close_Call"]}')
            price = float(self.optionsDF[self.optionsDF["StrikePrice"] == self.strikePrice]["Close_Call"])*t
            for i in range(len(x)):
                if x[i] <= self.strikePrice:
                    y[i] = price
                else:
                    price += s
                    y[i] = price
        else:
            price = float(self.optionsDF[self.optionsDF["StrikePrice"] == self.strikePrice]["Close_Put"])*t
            for i in range(len(x)-1,-1,-1):
                if x[i] >= self.strikePrice:
                    y[i] = price
                else:
                    price += s
                    y[i] = price
        # return y
        return np.add(y, np.array([Exchange_Fee for _ in range(len(x))]))
    def optionsScatter(self, x):
        return go.Scatter(
            x=x,
            y=self.optionsProfit(x),
            name = f"{self.BuySell}{self.PutCall}@{self.strikePrice}",
            visible = "legendonly",
            mode='lines',
            # line=go.scatter.Line(color='#6B99E5')
            opacity=0.4
        )


In [584]:
# x = np.arange(TAIEX_VALUE_BOTTOM-100, TAIEX_VALUE_TOP+200, 50)
# y = [0 for _ in range(len(x))]

# yc = np.add(optionsProfit(x, 11500, optionsDF, "call", "buy"), optionsProfit(x, 11550, optionsDF, "call", "sell"))
# yc = optionsProfit(x, 11550, optionsDF, "call", "buy")
# yp = np.add(optionsProfit(x, 11650, optionsDF, "put", "sell"), optionsProfit(x, 11700, optionsDF, "put", "buy"))

# fig = go.Figure(
#     data=[
#         go.Scatter(
#             x=x,
#             y=np.add(np.add(yc, yp), np.array([-4 for _ in range(len(x))])),
#             name = "Profit",
#             mode='lines',
#             # line=go.scatter.Line(color='#6B99E5')
#         ),
#         go.Scatter(
#             x=x,
#             y=yp,
#             name = "BuyCall@11500",
#             mode='lines',
#             line=go.scatter.Line(color='#ff79ff')
#         ),
#         # go.Scatter(
#         #     x=x,
#         #     y=optionsProfit(x, 11550, optionsDF, "call", "sell"),
#         #     name = "SellCall@11550",
#         #     mode='lines',
#         #     line=go.scatter.Line(color='#ffe500')
#         # )
#     ],
#     layout=go.Layout(
#         xaxis=plotly.graph_objects.layout.XAxis(
#             tickformat='d'
#         ),
#         yaxis=plotly.graph_objects.layout.YAxis(
#             tickformat='d'
#         )
#     )
#     )
# fig.show()

In [585]:
# x = np.arange(TAIEX_VALUE_BOTTOM-100, TAIEX_VALUE_TOP+200, 50)
# y = [0 for _ in range(len(x))]

# totalOtions = []
# totalOtions.append(options(11500, optionsDF, "call", "buy"))
# totalOtions.append(options(11550, optionsDF, "call", "sell"))
# # totalOtions.append(options(11700, optionsDF, "put", "buy"))
# # totalOtions.append(options(11650, optionsDF, "put", "sell"))
# # bc = options(11500, optionsDF, "call", "buy")
# # sc = options(11550, optionsDF, "call", "sell")
# # bp = options(11700, optionsDF, "put", "buy")
# # sp = options(11650, optionsDF, "put", "sell")

# fig = go.Figure()

# fig.add_trace(
#     go.Scatter(
#             x=x,
#             y=np.sum([op.optionsProfit(x) for op in totalOtions], axis=0),
#             name = f"Profit",
#             mode='lines'
#         ))
# # fig.add_trace(
# #     go.Scatter(
# #             x=x,
# #             y=np.sum([bc.optionsProfit(x), sc.optionsProfit(x), bp.optionsProfit(x), sp.optionsProfit(x)], axis=0),
# #             name = f"Profit",
# #             mode='lines'
# #         ))
# for op in totalOtions:
#     fig.add_trace(op.optionsScatter(x))    
# # fig.add_trace(bc.optionsScatter(x))
# # fig.add_trace(sc.optionsScatter(x))
# # fig.add_trace(bp.optionsScatter(x))
# # fig.add_trace(sp.optionsScatter(x))

# fig.update_layout(
#     title="Profit graph",
#     xaxis_title="x Axis Title",
#     yaxis_title="y Axis Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=12,
#         color="#7f7f7f"
#     ),
#     xaxis=plotly.graph_objects.layout.XAxis(
#         tickformat='d'
#     ),
#     yaxis=plotly.graph_objects.layout.YAxis(
#         tickformat='d'
#     )
# )
# fig.show()

In [586]:
x = np.arange(TAIEX_VALUE_BOTTOM-100, TAIEX_VALUE_TOP+200, 50)
y = [0 for _ in range(len(x))]

totalOtions = []
totalOtions.append(options(TAIEX_VALUE_BOTTOM-50, optionsDF, "call", "buy"))
totalOtions.append(options(TAIEX_VALUE_BOTTOM, optionsDF, "call", "sell"))
# totalOtions.append(options(TAIEX_VALUE_TOP, optionsDF, "put", "sell"))
# totalOtions.append(options(TAIEX_VALUE_TOP+50, optionsDF, "put", "buy"))


fig = go.Figure()

fig.add_trace(
    go.Scatter(
            x=x,
            y=np.sum([op.optionsProfit(x) for op in totalOtions], axis=0),
            name = f"Profit",
            mode='lines'
        ))

for op in totalOtions:
    fig.add_trace(op.optionsScatter(x))

fig.update_layout(
    title=f"Profit Graph Taiexs@{TAIEX_VALUE}",
    xaxis_title="x Axis Title",
    yaxis_title="y Axis Title",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    ),
    xaxis=plotly.graph_objects.layout.XAxis(
        tickformat='d'
    ),
    yaxis=plotly.graph_objects.layout.YAxis(
        tickformat='d'
    )
)
fig.show()

@@@@optionsDF
  Close_Call CallPut_Call  StrikePrice CallPut_Put Close_Put
0        171         Call      10600.0         Put        17
1         91         Call      10700.0         Put      35.5
2         56         Call      10750.0         Put        51
3         31         Call      10800.0         Put        75
4         14         Call      10850.0         Put       110
@@@@StrikePrice
10650


AttributeError: 'options' object has no attribute 'self'

In [ ]:
fileNameSettlementPrice = f'finalSettlementPrice.csv'
filePathSettlementPrice = f'/Users/singularity/Aaron/coding/fintech/{fileNameSettlementPrice}'

dfSettlementPrice = pandas.read_csv(filePathSettlementPrice)
dfSettlementPrice['Date'] = pandas.to_datetime(dfSettlementPrice['Date'], format='%Y/%m/%d')
dfSettlementPrice.sort_values(by='Date', inplace=True)
# dfSettlementPrice["Date"] = dfSettlementPrice["Date"].apply(lambda d:  d-datetime.timedelta(days=2))
dfSettlementPrice["Date"] = dfSettlementPrice.apply(lambda d:  d["Date"]-datetime.timedelta(days=2), axis=1)
dftt = pandas.merge(dfTaiexs, dfSettlementPrice, left_on=["Date"], right_on=["Date"]\
            , suffixes=('_Call','_Put'))[['Date', 'ClosePrice', 'TX/MTX', 'ContractMonthWeek']]
# , how="outer"
# dftt['Day_of_Week'] = dftt['Date'].dt.day_name()
# dftt
dftt.iloc[0]

In [ ]:
def WinLoss(dftt):
    # try:
        TAIEX_VALUE = float(dftt['ClosePrice'])
        TAIEX_VALUE_BOTTOM = int(TAIEX_VALUE - (TAIEX_VALUE % 50))
        TAIEX_VALUE_TOP = TAIEX_VALUE_BOTTOM + 50

        TX = float(dftt['TX/MTX'])
        

        DATE = dftt['Date']
        year = f'{DATE:%Y}'
        month = f'{DATE:%m}'
        day = f'{DATE:%d}'
        
        print(f'{year}/{month}/{day}')

        filename = f'optionsDailyInfo-{year}{month}{day}.csv'
        filepath = f'/Users/singularity/Aaron/coding/fintech/OptionsDailyInfoData/{filename}'
        df = pandas.read_csv(filepath)
        ContractMonthWeek = df['ContractMonthWeek'][0]
        StrikePriceLimite = df["StrikePrice"].apply(lambda d: True if int(d)>=TAIEX_VALUE-150 and int(d) <=TAIEX_VALUE+150 else False)
        dfCall = df[(df["CallPut"]=="Call") & (df["Session"]=="Regular") & (df["ContractMonthWeek"]==ContractMonthWeek) & StrikePriceLimite][['ContractMonthWeek', 'StrikePrice','CallPut', 'Close']]
        dfPut = df[(df["CallPut"]=="Put") & (df["Session"]=="Regular") & (df["ContractMonthWeek"]==ContractMonthWeek) & StrikePriceLimite][['ContractMonthWeek', 'StrikePrice','CallPut', 'Close']]
        optionsDF = pandas.merge(dfCall, dfPut, left_on=["StrikePrice", 'ContractMonthWeek'], right_on=["StrikePrice",      'ContractMonthWeek']\
                , suffixes=('_Call','_Put'))[['Close_Call', 'CallPut_Call', 'StrikePrice', 'CallPut_Put', 'Close_Put']]
        # optionsDF.astype({
        #     "Close_Call": "float64",
        #     "CallPut_Call": "string",
        #     "StrikePrice": "int64",
        #     "CallPut_Put": "string",
        #     "Close_Put": "float64"}, errors='ignore')
        x = np.arange(TAIEX_VALUE_BOTTOM-100, TAIEX_VALUE_TOP+200, 50)
        y = [0 for _ in range(len(x))]
        yc = [0 for _ in range(len(x))]
        yp = [0 for _ in range(len(x))]

        totalOtions = []
        totalOtions.append(options(TAIEX_VALUE_BOTTOM-50, optionsDF, "call", "buy"))
        totalOtions.append(options(TAIEX_VALUE_BOTTOM, optionsDF, "call", "sell"))
        totalOtions.append(options(TAIEX_VALUE_TOP, optionsDF, "put", "sell"))
        totalOtions.append(options(TAIEX_VALUE_TOP+50, optionsDF, "put", "buy"))
        
        for op in totalOtions:
            if op.PutCall == "call":
                yc = np.add(yc, op.optionsProfit(x))
            else:
                yp = np.add(yc, op.optionsProfit(x))
            y = np.add(y, op.optionsProfit(x))

        # y = np.sum([op.optionsProfit(x) for op in totalOtions], axis=0)
        # fig = go.Figure()

        # fig.add_trace(
        #     go.Scatter(
        #             x=x,
        #             y=np.sum([op.optionsProfit(x) for op in totalOtions], axis=0),
        #             name = f"Profit",
        #             mode='lines'
        #         ))

        # for op in totalOtions:
        #     fig.add_trace(op.optionsScatter(x))

        # fig.update_layout(
        #     title=f"Profit Graph Taiexs@{TAIEX_VALUE}",
        #     xaxis_title="x Axis Title",
        #     yaxis_title="y Axis Title",
        #     font=dict(
        #         family="Courier New, monospace",
        #         size=12,
        #         color="#7f7f7f"
        #     ),
        #     xaxis=plotly.graph_objects.layout.XAxis(
        #         tickformat='d'
        #     ),
        #     yaxis=plotly.graph_objects.layout.YAxis(
        #         tickformat='d'
        #     )
        # )
        # fig.show()

        if y.max() < 0 or yc.max() < 5 or yp.max() < 5:
            return 0
        i = 0
        while i <= len(y):
            if TX < x[i]:
                break
            i += 1
        j = i - 1
        if y[i]-y[j] == 0:
            return y[j]
        elif y[i]-y[j] > 0:
            return y[j]+(TX-x[j])
        else:
            return y[j]-(TX-x[j])
    # except Exception as e:
    #     print(e)
        # return None

In [ ]:
print(dftt.iloc[9])
WinLoss(dftt.iloc[9])

In [ ]:
dftt["WinLoss"] = dftt.apply(WinLoss, axis=1)

In [ ]:
dftt["WinLoss"].describe()

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
            x=dftt["Date"],
            y=dftt["WinLoss"],
            name = f"WinLoss",
            mode='lines'
        ))

fig.update_layout(
    title=f"Profit Graph Taiexs@{TAIEX_VALUE}",
    xaxis_title="x Axis Title",
    yaxis_title="y Axis Title",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    ),
    xaxis=plotly.graph_objects.layout.XAxis(
        tickformat='%Y-%m-%d'
    ),
    yaxis=plotly.graph_objects.layout.YAxis(
        tickformat='.2f'
    )
)
fig.show()